## Imports

In [15]:
import os

In [53]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import transformers
import re

In [11]:
#VARIABLES

SAMPLE_MODE = True

SUBMISSION_MODE = False

MODEL_PATH = 'model1'

## Load data and model

In [13]:
## LOAD DATA

if SAMPLE_MODE:
    n_rows=15000
else:
    n_rows=None

if not SUBMISSION_MODE :
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        %cd /content/drive/MyDrive/feedback-prize
        df = pd.read_csv('train_filled.csv',n_rows=n_rows,index_col=0)
    except:
        df = pd.read_csv('../raw_data/train_filled.csv',nrows=n_rows,index_col=0)
    display(df.head())
else : 
    print('WIP')

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
59951,0000D23A521A,1.617735e+12,0.0,170.0,"Some people belive that the so called ""face"" o...",Position,Position 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
59952,0000D23A521A,1.617735e+12,170.0,357.0,"It was not created by aliens, and there is no ...",Evidence,Evidence 1,34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 4...
59953,0000D23A521A,1.617735e+12,358.0,438.0,"A mesa is a naturally occuring rock formation,...",Evidence,Evidence 2,69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
59954,0000D23A521A,1.617735e+12,438.0,626.0,"This ""face"" on mars only looks like a face bec...",Claim,Claim 1,84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 9...
59955,0000D23A521A,1.617735e+12,627.0,722.0,Many conspiracy theorists believe that NASA is...,Counterclaim,Counterclaim 1,117 118 119 120 121 122 123 124 125 126 127 12...


In [10]:
## LOAD MODEL 

try: 
    model = tf.keras.models.load_model(MODEL_PATH)
except:
    print("Run this notebook with COLAB")

Run this notebook with COLAB


## Preprocess data

In [14]:
pd.read_csv("../raw_data/sample_submission.csv")

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [26]:
path = '../raw_data/test/'

X = pd.DataFrame(columns=['id','full_text'])

for file_name in os.listdir(path):
    with open(path+file_name,'r') as file:
        text = file.read()
        id_ = file_name.split('.')[0]
        X = X.append({'id':id_,'full_text':text},ignore_index=True)
        


In [33]:
# explode full_text into list
X['predictionstring'] = X['full_text'].apply(lambda text : list(range(len(text.split()))))
X['predictionstring'] = X['predictionstring'].apply(lambda nums : [str(n) for n in nums])
X['predictionstring'] = X['predictionstring'].apply(lambda nums : ' '.join(nums))

In [34]:
X

,id,full_text,predictionstring
0,DF920E0A7337,Have you ever asked more than one person for h...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,0FB0700DAF44,"During a group project, have you ever asked a ...",0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,D46BCB48440A,"When people ask for advice,they sometimes talk...",0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,18409261F5C2,80% of Americans believe seeking multiple opin...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,D72CB1C11673,Making choices in life can be very difficult. ...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [148]:
def get_essay_df(path):
    
    with open(path,'r') as file : 
        text = file.read()
        id_ = path.split('/')[-1].split('.')[0]
        X = pd.DataFrame({'id':id_,'text':text},index=[0])
        
    X['discourse'] = X['text'].apply(lambda text : text.split('.'))
    
    X = X.explode('discourse').drop('text',axis=1)
    
    X['discourse'] = X['discourse'].apply(lambda text : re.sub('^[\n]+','',text))
    X['discourse'] = X['discourse'].apply(lambda text : re.sub('[\n]+$','',text))
    
    X.reset_index(drop=True, inplace=True)
    
    X['discourse_len'] = X['discourse'].apply(lambda text :len(text.split()))
    
    X['predictionstring_start'] = X.discourse_len.cumsum().shift(+1).fillna(0).astype(int)
    X['predictionstring_end'] = X.discourse_len.cumsum()-1
        
    f = np.vectorize(lambda a,b : ' '.join(list(map(str,range(a,b+1)))),otypes=[str])
    X['predictionstring'] = f(X['predictionstring_start'],X['predictionstring_end'])
    
    X.drop(['discourse_len','predictionstring_start','predictionstring_end'],axis=1,inplace=True)

    
    return X

In [149]:
path = '../raw_data/test/'
file = os.listdir(path)[0]
X=get_essay_df(path+file)

In [150]:
X

,id,discourse,predictionstring
0,DF920E0A7337,Have you ever asked more than one person for h...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,DF920E0A7337,"Interviews are seen on the news, in professio...",29 30 31 32 33 34 35 36 37 38 39 40 41 42
2,DF920E0A7337,There are a lot of people that can help you b...,43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
3,DF920E0A7337,They can all open you up to new ideas,59 60 61 62 63 64 65 66 67
4,DF920E0A7337,I think talking to more than one person on an...,68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 8...
5,DF920E0A7337,"First, I think it shows you more than one option",108 109 110 111 112 113 114 115 116 117
6,DF920E0A7337,One opinion of a person might destroy your ch...,118 119 120 121 122 123 124 125 126 127 128 12...
7,DF920E0A7337,The whole point of asking what is the better ...,135 136 137 138 139 140 141 142 143 144 145 14...
8,DF920E0A7337,"You could already have the better opinion, an...",152 153 154 155 156 157 158 159 160 161 162 16...
9,DF920E0A7337,Multiple opinions can stride you toward diffe...,180 181 182 183 184 185 186 187
